In [2]:
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
%matplotlib inline

In [3]:
df = pd.read_csv('home_data.txt')

In [4]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
id               21613 non-null int64
date             21613 non-null object
price            21613 non-null int64
bedrooms         21613 non-null int64
bathrooms        21613 non-null float64
sqft_living      21613 non-null int64
sqft_lot         21613 non-null int64
floors           21613 non-null float64
waterfront       21613 non-null int64
view             21613 non-null int64
condition        21613 non-null int64
grade            21613 non-null int64
sqft_above       21613 non-null int64
sqft_basement    21613 non-null int64
yr_built         21613 non-null int64
yr_renovated     21613 non-null int64
zipcode          21613 non-null int64
lat              21613 non-null float64
long             21613 non-null float64
sqft_living15    21613 non-null int64
sqft_lot15       21613 non-null int64
dtypes: float64(4), int64(16), object(1)
memory usage: 3.5+ MB


In [8]:
features = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors','waterfront','view','condition','grade',
            'sqft_above','sqft_basement','yr_built','yr_renovated','zipcode','lat','long','sqft_living15','sqft_lot15']
target = ['price']

In [9]:
y=df[target]

In [10]:
X_train, X_test,Y_train,Y_test = train_test_split(df[features] ,y ,test_size=0.2)

In [11]:
print X_train.shape, Y_train.shape
print X_test.shape, Y_test.shape

(17290, 18) (17290, 1)
(4323, 18) (4323, 1)


## Now we want a working regression model

In [16]:
feature_lm = linear_model.LinearRegression()

In [17]:
feature_model = feature_lm.fit(X_train,Y_train)

In [18]:
target = feature_lm.predict(X_test)

## Calculate R squared

### So 69% of variability in the Y can be explained by 

In [19]:
print('Linear Regression R squared: %.4f' % feature_lm.score(X_test,Y_test))

Linear Regression R squared: 0.6941


## Calculate RMSE

### Our model is able to predict the value os every house within $204351 of the real price

In [23]:
import numpy as np
from sklearn.metrics import mean_squared_error
lin_mse = mean_squared_error(feature_predictions, Y_test)
lin_rmse = np.sqrt(lin_mse)
print('Linear Regression RMSE: %.4f' % lin_rmse)

Linear Regression RMSE: 204351.4751


## Calculate mean absolute error (MAE)

In [24]:
from sklearn.metrics import mean_absolute_error
lin_mae = mean_absolute_error(feature_predictions, Y_test)
print('Linear Regression MAE: %.4f' % lin_mae)

Linear Regression MAE: 125350.9750


# Let’s try a more complex model to see whether results can be improved 

## RandomForestRegressor 

In [25]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(random_state=42)
forest_reg.fit(X_train, Y_train)

C:\Users\shivasupraj\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [29]:
print('Random Forest R squared: %.4f' % forest_reg.score(X_test, Y_test))

Random Forest R squared: 0.8469


In [30]:
feature_predictions = forest_reg.predict(X_test)
forest_mse = mean_squared_error(feature_predictions, Y_test)
forest_rmse = np.sqrt(forest_mse)
print('Random Forest RMSE: %.4f' % forest_rmse)

Random Forest RMSE: 144553.2402


## Gradient boosting

In [31]:
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingRegressor
model = ensemble.GradientBoostingRegressor()
model.fit(X_train, Y_train)

C:\Users\shivasupraj\Anaconda2\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [33]:
print('Gradient Boosting R squared: %.4f' % model.score(X_test, Y_test))

Gradient Boosting R squared: 0.8602


In [34]:
feature_predictions = model.predict(X_test)
model_mse = mean_squared_error(feature_predictions, Y_test)
model_rmse = np.sqrt(model_mse)
print('Gradient Boosting RMSE: %.4f' % model_rmse)

Gradient Boosting RMSE: 138163.9397


# Select only the features that are important

In [46]:
feature_label = np.array(features)

In [47]:
feature_label

array(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat',
       'long', 'sqft_living15', 'sqft_lot15'], 
      dtype='|S13')

In [57]:
important_features = model.feature_importances_

In [58]:
feature_indexes_by_importance = important_features.argsort()
for index in feature_indexes_by_importance:
    print('{} - {:.2f}%'.format(feature_label[index], (important_features[index] *100.0)))

floors - 0.15%
bedrooms - 0.78%
sqft_lot15 - 1.75%
yr_renovated - 1.78%
bathrooms - 2.17%
condition - 2.30%
sqft_basement - 2.62%
view - 2.75%
sqft_above - 3.49%
waterfront - 4.25%
sqft_lot - 4.32%
yr_built - 5.32%
zipcode - 6.08%
sqft_living15 - 7.72%
grade - 10.08%
long - 10.22%
sqft_living - 11.63%
lat - 22.58%
